# Chaper 21. Class Metaprogramming

    [Metaclass] are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with cetaintly that they  need them, and don't need a explaination about why).
    
    -- Tim Peters

Class metaprogramming is the art of creating or customizing classes at runtime. Classes are first-class object in Python, so a function can be used to create a new class at any time, without using the class keyword. Class decorators are also functions, but capable of inspecting, changing, and even replacing the decorated class with another class. Finnaly, metaclass are the most advanced tool for class metaprogramming; they let you create whole new categories of classes with special traits, such as the abstract base classes we've already seen.

Metaclass are powerful, but hard to get right. Class decorators solve many of the same problems more simply.

## A Class Factory

The standard library has class factory that we've seen several times  in this book: collections.nametuple. It's a function that, given a class name and attribute names creates a subclass of tuple that allows retrieving items by name and provides a nice `__repr__` for debuggig.

In [1]:
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass
    field_names = tuple(field_names)
    
    def __init__(self, *args, **kwags):
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwags)
        
        for name, value in attrs.items():
            setattr(self, name, value)
            
    def __iter__(self,):
        for name in self.__slots__:
            yield getattr(self, name)
            
    cls_attrs = dict(__slots__ = field_names,
                     __init__ = __init__,
                     __iter__ = __iter__)
    return type(cls_name, (object,), cls_attrs)

In [2]:
Dog = record_factory('Dog', 'name weight owner')

In [4]:
dog = Dog(name='Rex', weight=30, owner='Bob')

In [6]:
dog.name, dog.weight

('Rex', 30)

In [7]:
name, weight, _ = dog

In [8]:
name, weight

('Rex', 30)

In summary, the last line of `record_factory` builds a class named by the vlaue of cls_name, with objects as its single superclass and with  class attributes named `__slots__`, `__init__` .

> It's good practice to avoid exec or eval for metaporamming in Python. These functions pose serios security risks if they are fed string from untrusted sources.

**Instances of classes created by `record_factory` have a limitation: they are not serializable - that is, they can't be used with the dump/load function from the pickle module.**

## What Happens When: Import Time Versus Runtime

For successful metaprogramming, you must be aware of when the Python interpreter evaluates each block of code. Python programmers talk about "import time" versus "runtime" but the terms are not strictly defined and there are a gray area between them. At import time, the interpreter parses the source code of .py module in one pass from top to bottom, and generates the bytecode to be executed. That's when syntax errors may occur. If there is an up-to-date .pyc file available in the local `__pycache__`, those steps are skipped because the bytecode is ready to run.

The interpreter executes a def statement on the top level of a module when the module is imported, but what does that archieve? The interpreter compiles the function body, and binds the function object to its global name, but it does not execute the body of the function, obviously. In the usual case, this means that the interpreter defines top-level functions at import time, but executes their bodies only when the functioins are  invoked at runtime.

For classes, the story is different: at import time, the interpreter executes the body of every class, even the body of classes nested in other classes. Execution of a class body means that the attributes and methods of the class are defined, and then the class object itself is built. In this sense, the body of classes is "top-level code": it runs at import time.

## Metaclasses 101

A metaclass is a class factory, except that instead of a function. A metaclass is a class that builds classes.

**Consider the Python object model: classes are objects, thereofore each class must be an intance of some other class. By default, Python classes are instance of type. In other words, `type` is the metaclass for most built-in and user-defined classes.**

To avoid infinite regress, `type` is an instance of itself.

In [1]:
type(type)

type

In [2]:
type.__mro__

(type, object)

In [3]:
type(object)

type

In [4]:
object.__mro__

(object,)

> The classes object and type have a unique relationship: object is an instance of type, and type is subclass of object. This relationship is "magic": it cannot be expressed in Python because either class would have to exist before the other could be defined. The fact that type is an instance of itself is also magical.

The important takeaway here is that all classes are instances of type, but metaclass are subclass of type, so they act as class factories. In particular, a metaclass can customize its instance by implementing `__init__`. A metaclass `__init__` method can do everything a class decorator can do.

## The Metaclass `__prepare__` Special Method

In some application it's interesting to be able to know the order in which the attributes of a class are defined.

As we've seen, both the type constructor and the `__new__` and `__init__` methods of metaclass receive the body of the class evaluated as a mapping of names to attributes. However, by default, that mapping is a dict, which means the order of the attributes as they appear in the class body is lost by the time our metaclass or class decorator can look at them.

The solution to this problem is the `__prepare__` special method, introduced in Python 3. The special method is relevant only in metaclasses, and it must be a class method. The `__prepare__` method is invoked by the interpreter before the `__new__` method in the metaclass to create the mapping that will be filled with attributes from the class body. Besides the metaclass as fist argument, `__prepare__` gets the name of the class to be constructed and its tuple of base classes, and it must return a mapping, which will be received s the last argument by `__new__` and then `__init__` when the metaclass builds a new class.

In [5]:
import collections

In [6]:
class Validated: pass

In [7]:
class EntityMeta(type):
    """Metaclass for business entities with validated fields"""
    
    @classmethod
    def __prepare__(cls, name, bases):
        # Return an empty OrderDict instance, where the class attributes
        # will be stored
        return collections.OrderedDict()
    
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        # Create a _field_names attributes in the class under contruction
        cls._field_names = []
        
        # This line is unchanged from the previous version, but attr_dict
        # here is the OrderDict obtained by the interpreter when it called
        # __prepare__ before calling __init__. Therefore, this for loop
        # will go over the attributes in the order they were added.
        for key, attr in attr_dict.items():
            if isinstance(attr, Validated):
                type_name = type(attr).___name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)

In [10]:
class Entity(metaclass=EntityMeta):
    """Business entity with validated fields"""
    
    @classmethod
    def field_names(cls):
        # The field_names class method simply yields the names of the
        # fields in the order they were added.
        for name in cls._field_names:
            yield name

In the readl world, metaclass are uesd in frameworks and libraries that help programmers perform:

* Attribute validation
* Applying decorators to many methods at once
* Object serialization or data conversion
* Object-relational mapping
* Object-based persistency
* Dynamic translation of class structures from other languages

## Classes as Objects

Every class has a number of attribtues defined in the Python data model. Three of these attributes we've seen several times in the book already: `__mro__`, `__class__`, and `__name__`. Other class attributes are:

`cls.__bases__`

    The tuple of base classes of the class.
    
`cls.__qualname__`

    A new attribute in Python 3.3 holding the qualified name of a class or function, which is a dotted path from the global scope of the module to the class definition.
    
`cls.__subclasses__()`:

    This method returns a list of the immediate sublass of the class. The implementation uses weak references to avoid circular references between the superclass and its subclasses - which holds a strong referencesto the superclasses in their __bases__ attribute. The method returns the list of subclass that currently exist in memory.
    
`cls.mro()`

    The interpreter calls this method when building a class to obtain the tuple of superclass that is stored in the __mro__ attribute of the class. A metaclass can override this method to customize the method resolution order of the clas under constructin.